# Setting up Paths

In [15]:
#path to the data
pathCOMPASOutput = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'

#path to compasrepo which contains ther popsynth folder
pathRepo = '/home/cneijssel/Documents/COMPASpop'



# Imports

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import pandas as pd
import sys
import os.path
#Quick fudge to make import from ../Scripts work
import sys
sys.path.append(pathRepo + '/popsynth/Papers/NeijsselEtAL/CosmicIntegration/Scripts')
sys.path.append(pathRepo + '/popsynth/Papers/NeijsselEtAL/CosmicIntegration/Scripts/CosmicScripts')

# Custom Imports from COMPASrepo

In [17]:
import ClassCosmicLoop as Cosmic #Given settings and redshifts returns rates (2D arrays) Loads the data
import coencodeVarious as CV


# COMPAS info

In [18]:
f               = h5.File(pathCOMPASOutput+'COMPASOutput.h5')
fDCO            = f['doubleCompactObjects']
metallicityGrid = np.unique(f['systems']['Metallicity1'][...].squeeze())
def returnDCOmask(types='BBH', optimistic=True):
    if types == 'BBH':
        maskDCO   = (fDCO['stellarType1'][...] == 14) & (fDCO['stellarType2'][...] == 14) &\
                    (fDCO['RLOFSecondaryAfterCEE'][...]==False) &(fDCO['mergesInHubbleTimeFlag'][...]==True)
    if types == 'BNS':
        maskDCO   = (fDCO['stellarType1'][...] == 13) & (fDCO['stellarType2'][...] == 13) &\
                    (fDCO['RLOFSecondaryAfterCEE'][...]==False) &(fDCO['mergesInHubbleTimeFlag'][...]==True)
    if types == 'BHNS':
        maskDCO1   = (fDCO['stellarType1'][...] == 13) & (fDCO['stellarType2'][...] == 14) &\
                    (fDCO['RLOFSecondaryAfterCEE'][...]==False) &(fDCO['mergesInHubbleTimeFlag'][...]==True)
        maskDCO2   = (fDCO['stellarType1'][...] == 14) & (fDCO['stellarType2'][...] == 13) &\
                    (fDCO['RLOFSecondaryAfterCEE'][...]==False) &(fDCO['mergesInHubbleTimeFlag'][...]==True)
        maskDCO    = maskDCO1 | maskDCO2


    if not optimistic:
        maskPessimistic = (fDCO['optimisticCEFlag'][...]==False)
        maskDCO = maskDCO & maskPessimistic
    return maskDCO

#remember that between different MSSFRs the DCO used are exactly the same only the weights differ.
#so for this array we do not have to do the integral just now which DCO's we are looking at :)
types          = 'BBH'
optimistic     = False
maskDCO        = returnDCOmask(types=types, optimistic=optimistic)
seedsDCO       = fDCO['seed'][...].squeeze()[maskDCO.squeeze()]


In [19]:
#I want to do it once and store booleans in array
#careful this is not a univeral approach  if stuff changed it brakes
filename = 'BooleanArray.txt'


columnnames = ['PPISN_primary' ,\
          'PPISN_secondary',\
          'PISN_primary',\
          'PISN_secondary',\
          'LBV_primary',\
          'LBV_secondary',\
          'channel_binary']

header = ''
for string in columnnames[:-1]:
    header +=string+'\t'
header += columnnames[-1]
if not os.path.isfile(filename):
    print(' does not file exists')
   
    f_handle = file(filename, 'w')
    np.savetxt(f_handle, booleanArray, delimiter='\t', fmt="%5i", header=header, comments='')
    f_handle.close()
else:
    print('file exists')


file exists


# (P)PISN estimate flag

In [24]:
doPPISN = True

if doPPISN:
     #table 1 woosley 2016 
    PPISN_CO_low  =  28.
    PISN_CO_low   =  54.
    
    seedsSN       =  f['supernovae']['randomSeed'][...].squeeze()
    boolSN_DCO    =  np.in1d(seedsSN,seedsDCO) #will be twice length seedsDCO (2 SN)
    seedsSNtest      =  f['supernovae']['randomSeed'][...].squeeze()[boolSN_DCO]
    print len(np.unique(seedsSNtest)), len(seedsDCO)
    #have a look at the primaries
    #apperantly I have two systems of which both went supernova simultaneously in code
    #not sure how this is printed, but affects 2/291929 systems 
    #problem is that I cannot assign it easily to DCO
    #or statement in whichstar results in double usage implyin itSN is printed twice if both explode
    #and statement is not possible.
    #need to isolate the seed and manually set it
    
    doubleSN      =  (f['supernovae']['whichStar'][...].squeeze() == 3)[boolSN_DCO]
    doublePPISN   = ((f['supernovae']['MassCOCoreSN'][...].squeeze()>=PPISN_CO_low) &\
                     (f['supernovae']['MassCOCoreSN'][...].squeeze()< PISN_CO_low))[boolSN_DCO][doubleSN]
    doublePISN      = (f['supernovae']['MassCOCoreSN'][...].squeeze()>= PISN_CO_low)[boolSN_DCO][doubleSN]
    print 'which star == 3 PPISN/PISN=', np.unique(doublePPISN), np.unique(doublePISN)
    
    #so I know that these systems should have their boolean false, problem is their location
    #instead of directly using primaryPPISN boolean i need to convert to seed and lookup seeds of DCO.
    
    boolPrimary   =  (f['supernovae']['whichStar'][...].squeeze() == 1)[boolSN_DCO]  
    
    primaryPPISN    = ((f['supernovae']['MassCOCoreSN'][...].squeeze()>=PPISN_CO_low) &\
                     (f['supernovae']['MassCOCoreSN'][...].squeeze()< PISN_CO_low))[boolSN_DCO][boolPrimary]
    primaryPISN      = (f['supernovae']['MassCOCoreSN'][...].squeeze()>= PISN_CO_low)[boolSN_DCO][boolPrimary]
        
    seedsPrimaryPPISN = seedsSN[boolSN_DCO][boolPrimary][primaryPPISN]
    seedsPrimaryPISN  = seedsSN[boolSN_DCO][boolPrimary][primaryPISN]
    
        #have a look at the secondaries
    boolSecondary =  (f['supernovae']['whichStar'][...].squeeze() == 2)[boolSN_DCO]
    secondaryPPISN     = ((f['supernovae']['MassCOCoreSN'][...].squeeze()>=PPISN_CO_low) &\
                    (f['supernovae']['MassCOCoreSN'][...].squeeze()< PISN_CO_low))[boolSN_DCO][boolSecondary]
    secondaryPISN      = (f['supernovae']['MassCOCoreSN'][...].squeeze()>= PISN_CO_low)[boolSN_DCO][boolSecondary]
        
    seedsSecondaryPPISN = seedsSN[boolSN_DCO][boolSecondary][secondaryPPISN]
    seedsSecondaryPISN  = seedsSN[boolSN_DCO][boolSecondary][secondaryPISN]    
    #read and write to file
    df = pd.read_table(filename, header=0,  delim_whitespace=True)
    df['PPISN_primary']   = np.in1d(seedsDCO, seedsPrimaryPPISN)
    df['PPISN_secondary'] = np.in1d(seedsDCO, seedsPrimaryPISN)
    df['PISN_primary']    = np.in1d(seedsDCO, seedsPrimaryPISN)
    df['PISN_secondary']  = np.in1d(seedsDCO, seedsSecondaryPISN)
    
    Results = df.as_matrix()
    
    f_handle = file(filename, 'w')
    np.savetxt(f_handle, Results, delimiter='\t', fmt='%5i', header=header, comments='')
    f_handle.close()
#If a PISN it would not be a binary so sepatate class

291929 291929
which star == 3 PPISN/PISN= [False] [False]
244 0


# LBV estimate flag

In [7]:
doLBV  = False

#During the run the lbv flag didnt exist, now estimate from sse grid if the star is above lbv threshold.
#first star will only be from zams mass, unless there is an
#extreme second moment mass transfer from winds/timestepping, but I ignore this
#Secondary star can accrete during the MS so I test the MS after mass transfer

#this takes long :s
if doLBV:
    metallicities = np.logspace(-4,-1.5,30)
    LBVMS         = np.zeros(len(metallicities))
    masses        = np.linspace(0.0,200,401)
    for nrZ, Z in enumerate(metallicities):
        path= '/home/cneijssel/Desktop/GridStuff/'+str(Z)
        dfDetailed = pd.read_table(path+'/dataOutput_0.dat', header=1, delim_whitespace=True)
        boolLBV    = (dfDetailed['m_LBVphaseFlag']==True)
        boolMS     = (dfDetailed['S1type']==True)

        seedsS     = np.unique(dfDetailed['%SEED'][boolLBV & boolMS])
        boolM      = np.zeros(len(masses), dtype=bool)

        for S in (seedsS):
            boolM[S] = True
        try:
            minMassLBV = min(masses[boolM])    
        except: #no LBV so arbitrary high mass
            minMassLBV = 1000.
        LBVMS[nrZ] =minMassLBV
    print 'found LBV criteria looking at booleans'   
    #split cell because this one takes long if a bug below I dont want to redo it

In [8]:
if doLBV:
    print 'found LBV criteria looking at booleans'
    boolLBVprimary   = np.zeros(len(f['systems']['mass1'][...].squeeze()), dtype=bool)
    boolLBVsecondary = np.zeros(len(f['systems']['mass1'][...].squeeze()), dtype=bool)
    for nrZ, Z in enumerate(metallicityGrid):

        boolZ     = f['systems']['Metallicity1'][...].squeeze() == Z
        boolMass1 = f['systems']['mass1'][...].squeeze() >= LBVMS[nrZ]
        boolLBV   = boolZ & boolMass1
        boolLBVprimary = boolLBV | boolLBVprimary
        boolMass2 = f['systems']['mass2'][...].squeeze() >= LBVMS[nrZ]
        boolLBV   = boolZ & boolMass2
        boolLBVsecondary = boolLBV | boolLBVsecondary
    seedsPrimaryLBV = f['systems']['SEED'][...].squeeze()[boolLBVprimary]
    seedsSecondaryLBV = f['systems']['SEED'][...].squeeze()[boolLBVsecondary]
    print len(seedsPrimaryLBV)

    boolSecondaryLBVRLOF = np.zeros(len(f['RLOF']['mass1'][...].squeeze()), dtype=bool)
    for nrZ, Z in enumerate(metallicityGrid):
        #find metallicity Z
        boolZ     = f['systems']['Metallicity2'][...].squeeze() == Z
        seedsZ = f['systems']['SEED'][...].squeeze()[f['systems']['Metallicity1'][...].squeeze() ==  Z]
        #get seeds with metallicity Z
        boolZ = np.in1d(f['RLOF']['randomSeed'][...].squeeze(), seedsZ)
        boolMass = f['RLOF']['mass2'][...].squeeze() >= LBVMS[nrZ]
        boolType = f['RLOF']['type2'][...].squeeze() == 1
        boolLBV = boolMass & boolType
        boolSecondaryLBVRLOF = boolLBV | boolSecondaryLBVRLOF
    seedsSecondaryLBVRLOF = f['RLOF']['randomSeed'][...].squeeze()[boolSecondaryLBVRLOF]
    print 'almost DOnce'
    boolLBV_Primary = np.in1d(seedsDCO, seedsPrimaryLBV)
    boolLBV_Secondary = np.in1d(seedsDCO, seedsSecondaryLBV) |  np.in1d(seedsDCO, seedsSecondaryLBVRLOF)
    #read and write to file
    df = pd.read_table(filename, header=0,  delim_whitespace=True)
    df['LBV_primary']   = boolLBV_Primary
    df['LBV_secondary'] = boolLBV_Secondary
    
    Results = df.as_matrix()
    
    f_handle = file(filename, 'w')
    np.savetxt(f_handle, Results, delimiter='\t', fmt='%5i', header=header, comments='')
    f_handle.close()
    print 'Done'

# formation flag

In [14]:
doChannels = True
#when calculating the fractions I saved the seeds to file to reuse here.
if doChannels:
    channel1 = pd.read_table('../AppendixE_Fractions/seedsClassic.txt', header=None)
    maskChannel1 = np.in1d(seedsDCO, channel1.as_matrix().squeeze())
    channel2 = pd.read_table('../AppendixE_Fractions/seedsNonCee.txt', header=None)
    maskChannel2 = np.in1d(seedsDCO, channel2.as_matrix().squeeze())
    channel3 = pd.read_table('../AppendixE_Fractions/seedsDoubleCore.txt', header=None)
    maskChannel3 = np.in1d(seedsDCO, channel3.as_matrix().squeeze())

    df = pd.read_table(filename, header=0,  delim_whitespace=True)
    df['channel_binary'][maskChannel1]  = 1
    df['channel_binary'][maskChannel2]  = 2
    df['channel_binary'][maskChannel3]  = 3
    for channel in [maskChannel1, maskChannel2, maskChannel3]:
        print np.sum(channel) / float(len(channel))
    Results = df.as_matrix()
    
    
    f_handle = file(filename, 'w')
    np.savetxt(f_handle, Results, delimiter='\t', fmt='%5i', header=header, comments='')
    f_handle.close()

0.500745044172
0.395688677726
0.0685098088919
